In [3]:
import configparser
import os
import sys
from os import path

PATH_ROOT = ""
PATH_DATA = ""

creds_path_ar = ["../../credentials.ini", "credentials.ini"]

for creds_path in creds_path_ar:
    if path.exists(creds_path):
        config_parser = configparser.ConfigParser()
        config_parser.read(creds_path)
        PATH_ROOT = config_parser['MAIN']["PATH_ROOT"]
        PATH_DATA = config_parser['MAIN']["PATH_DATA"]
        WANDB_enable = config_parser['MAIN']["WANDB_ENABLE"] == 'TRUE'
        ENV = config_parser['MAIN']["ENV"]

# adding cwd to path to avoid "No module named src.*" errors
sys.path.insert(0, os.path.join(PATH_ROOT))

In [56]:
import argparse
import random
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim
from sklearn.metrics import roc_curve, auc
from src.data import get_data
from src.data.iterable_dataset import Config, DataDict, StreamingDataset, iq_to_spectogram, \
    normalize
from src.models import arch_setup, tcn_model3
from src.data import get_data
from src.visualization import metrics
from src.features import specto_feat
import wandb
import logging


In [30]:
cd {PATH_ROOT}
wandb.init()

/home/ubuntu/sota-mafat-radar


In [71]:
# import importlib
# importlib.reload(metrics)

<module 'src.visualization.metrics' from '/home/ubuntu/sota-mafat-radar/src/visualization/metrics.py'>

# Two Parts

1. Run off the test data to get the test scores for a given model (to give us some indication of the accuracy)
2. With the various models, create a LR using the test data as its own train/val.

### PART 1

In [76]:
def load_model(model_path: str):
    '''
    Load Model from Wandb
    '''
    wandb.restore('data/models/model.pth', run_path=model_path)
    return torch.load('data/models/model.pth')


def load_testset(test_path: str):
    '''
    Load Test Data
    '''
    test_data = pd.DataFrame.from_dict(get_data.load_data(test_path, PATH_DATA), orient='index').transpose()
    test_data['target_type'].replace({'animal': 0, 'human': 1}, inplace=True)

    return test_data


def run_predictions(model, test_df):
    '''
    Have the predictions ready for submission
    '''
    test_df['output_array'] = test_df['iq_sweep_burst'].progress_apply(iq_to_spectogram)
    test_df['output_array'] = test_df.progress_apply(lambda row: specto_feat.max_value_on_doppler(row['output_array'], row['doppler_burst']), axis=1)
    test_df['output_array'] = test_df['output_array'].progress_apply(normalize)
    test_x = torch.from_numpy(np.stack(test_df['output_array'].tolist(), axis=0).astype(np.float32)).unsqueeze(1)

    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu:0')

    # Creating DataFrame with the probability prediction for each segment
    submission = pd.DataFrame()
    submission[['segment_id', 'label']] = test_df[['segment_id', 'target_type']]
    submission['prediction'] = model(test_x.to(device)).detach().cpu().numpy()
    submission['label'] = test_df['target_type']
    return submission, test_df['target_type']

def check_model_auc(model_path: str, test_path: str):
    '''
    1. Load the Model (using load_model())
    2. Load the Test Data (using load_testdata())
    3. Return the predictionsauc and acc scores of predictions
    '''
    model = load_model(model_path)
    test_df = load_testset(test_path)
    predictions, labels = run_predictions(model, test_df)
    return metrics.model_scores(labels, predictions['prediction'])

In [78]:
# model_path = 'sota-mafat/sota-mafat-base/3s0bv1dr'
# model = load_model(model_path)
# test_path = 'MAFAT RADAR Challenge - FULL Public Test Set V1'
# test_dict = load_testset(test_path)
# predics = run_predicions(model, test_path)

In [77]:
check_model_auc(model_path,test_path)

100%|██████████| 284/284 [00:00<00:00, 13200.74it/s]


0.6203593408775064

In [79]:
model_paths = ['sota-mafat/sota-mafat-base/1epmi6lf','sota-mafat/sota-mafat-base/3s0bv1dr']

### PART 2

In [89]:
preds = [pred1['prediction'],pred2['prediction']]
pd.concat(preds, axis=1)

,prediction,prediction
0,0.887834,0.887834
1,0.902009,0.902009
2,0.902022,0.902022
3,0.906737,0.906737
4,0.840028,0.840028
...,...,...
279,0.921163,0.921163
280,0.889626,0.889626
281,0.916283,0.916283
282,0.916751,0.916751


In [ ]:
def lr_model(model_paths: list, test_path):
    preds = []
    col_names = range(len(model_paths))
    test_df = load_testset(test_path)
    for model_path in model_paths:
        model = load_model(model_path)
        pred,_ = run_predictions(model, test_df)
        preds.append(preds['prediction'])
    df = pd.concat(preds, axis=1)
    df.columns = col_names 